In [1]:
# 45 Page
# 데이터 구성:Series, DataFrame
import pandas as pd
# 행렬 연산
import numpy as np
# 데이터 시각화
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
# 선형 회귀모델 formula(y ~ X1 + X2 + ...)
import statsmodels.formula.api as smf
# 잔차의 정규성 확인 :qqplot, 회귀분석의 상수항 추가 : add_constant
from statsmodels.api import qqplot, add_constant
# 다중공선성 확인
from statsmodels.stats.outliers_influence import variance_inflation_factor
# 학습용/평가용 데이터 분리:train, test
from sklearn.model_selection import train_test_split
# 선형회귀모형
from sklearn.linear_model import LinearRegression
# 변수 선택법(후진제거법)
from sklearn.feature_selection import RFE
# Scale 변환->표준화 회귀계수 산출
from sklearn.preprocessing import StandardScaler
# 평가함수
from sklearn.metrics import r2_score

In [2]:
from sklearn.metrics import confusion_matrix, classification_report

In [3]:
# 그래프 한글폰트 적용:맑은 고딕
matplotlib.rc("font", family = "NanumGothic")
# 그래프 (-) 기호 표시
matplotlib.rc("axes", unicode_minus = False)

In [9]:
import pandas as pd
df_raw = pd.read_csv("df_model.csv")
df_raw.head(21)

,Date,rate_delay,rate_female,rate_2030s,target
0,2021-01-01,0.024611,0.560233,0.664508,1303250.0
1,2021-01-02,0.037942,0.604011,0.604011,22230200.0
2,2021-01-03,0.000000,0.794872,0.512821,10492740.0
3,2021-01-04,0.038515,0.625058,0.670534,18678110.0
4,2021-01-05,0.025445,0.592875,0.676845,19482900.0
5,2021-01-06,0.037942,0.604011,0.604011,11861650.0
6,2021-01-07,0.047674,0.644186,0.713953,11432100.0
7,2021-01-08,0.028869,0.597434,0.725742,2023290.0
8,2021-01-09,0.037942,0.604011,0.604011,17727520.0
9,2021-01-10,0.000000,0.666667,0.619048,11129340.0


In [10]:
df = df_raw.drop("Date",axis = 1)
df

,rate_delay,rate_female,rate_2030s,target
0,0.024611,0.560233,0.664508,1303250.0
1,0.037942,0.604011,0.604011,22230200.0
2,0.000000,0.794872,0.512821,10492740.0
3,0.038515,0.625058,0.670534,18678110.0
4,0.025445,0.592875,0.676845,19482900.0
...,...,...,...,...
1090,0.040787,0.579024,0.756009,NaN
1091,0.026260,0.606813,0.665720,NaN
1092,0.032045,0.592837,0.715363,NaN
1093,0.000000,0.606383,0.680851,NaN


In [19]:
df.loc[730:1095, :]

,rate_delay,rate_female,rate_2030s,target
730,0.047312,0.605376,0.754301,NaN
731,0.040286,0.589973,0.696509,NaN
732,0.065312,0.620464,0.648041,NaN
733,0.026718,0.628499,0.685751,NaN
734,0.042598,0.562849,0.675978,NaN
...,...,...,...,...
1090,0.040787,0.579024,0.756009,NaN
1091,0.026260,0.606813,0.665720,NaN
1092,0.032045,0.592837,0.715363,NaN
1093,0.000000,0.606383,0.680851,NaN


In [20]:
df_prediction =df.loc[730:1095, :]
df = df.drop(df_prediction.index)
df

,rate_delay,rate_female,rate_2030s,target
0,0.024611,0.560233,0.664508,1303250.0
1,0.037942,0.604011,0.604011,22230200.0
2,0.000000,0.794872,0.512821,10492740.0
3,0.038515,0.625058,0.670534,18678110.0
4,0.025445,0.592875,0.676845,19482900.0
...,...,...,...,...
725,0.035917,0.583176,0.662571,22032270.0
726,0.035055,0.564576,0.736162,22423860.0
727,0.030899,0.594569,0.729401,16659780.0
728,0.051402,0.487150,0.724299,1479180.0


In [21]:
df_prediction

,rate_delay,rate_female,rate_2030s,target
730,0.047312,0.605376,0.754301,NaN
731,0.040286,0.589973,0.696509,NaN
732,0.065312,0.620464,0.648041,NaN
733,0.026718,0.628499,0.685751,NaN
734,0.042598,0.562849,0.675978,NaN
...,...,...,...,...
1090,0.040787,0.579024,0.756009,NaN
1091,0.026260,0.606813,0.665720,NaN
1092,0.032045,0.592837,0.715363,NaN
1093,0.000000,0.606383,0.680851,NaN


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 730 entries, 0 to 729
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   rate_delay   730 non-null    float64
 1   rate_female  730 non-null    float64
 2   rate_2030s   730 non-null    float64
 3   target       730 non-null    float64
dtypes: float64(4)
memory usage: 22.9 KB


In [23]:
# df= df.rename(columns = {"남성":"MEN","여성":"WOMEN", "4060s": "s_4060", "2030s":"s_2030","여성_비율":"WOMEN_RATIO","s_2030 비율" :"s_2030_ratio"})
# df

In [24]:
# train/test data 분리
df_train, df_test = train_test_split(df, test_size = 0.3) 
print("train data size : {}".format(df_train.shape))
print("test data size : {}".format(df_test.shape))

train data size : (511, 4)
test data size : (219, 4)


In [25]:
# 선형 회귀 분석 : formula(y ~ X1 + X2 + ...) 이용. 상수항(Intercept) 자동 추가.
reg_model = smf.ols(formula = "target ~ rate_delay + rate_female + rate_2030s", data = df_train)
# 적합
reg_result = reg_model.fit()
print(reg_result.summary())

                            OLS Regression Results                            
Dep. Variable:                 target   R-squared:                       0.007
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     1.259
Date:                Thu, 21 Mar 2024   Prob (F-statistic):              0.288
Time:                        00:13:59   Log-Likelihood:                -8800.1
No. Observations:                 511   AIC:                         1.761e+04
Df Residuals:                     507   BIC:                         1.763e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept    1.209e+07   4.42e+06      2.737      

In [26]:
# 설명변수간의 다중공선성 확인 : 설명변수 데이터 생성 및 상수항 추가
df_train_x = df_train.drop('target', axis = 1)
# statsmodels의 상수항 추가 함수 적용
df_train_x_const = add_constant(df_train_x) 
# DataFrame으로 저장
df_vif = pd.DataFrame()
df_vif["variable"] = df_train_x_const.columns
df_vif["VIF"] = [variance_inflation_factor(df_train_x_const.values, i) \
                 for i in range(df_train_x_const.shape[1])]
# VIF 정렬
df_vif.sort_values("VIF", ascending = True).round(2)

,variable,VIF
2,rate_female,1.01
3,rate_2030s,1.02
1,rate_delay,1.02
0,const,186.16


In [27]:
# 선형 회귀 분석(유의하지 않은 변수 중 p-value가 가장 큰 것을 하나씩 반복적으로 제거)
# rate_female 제거
reg_model = smf.ols(formula = "target ~ rate_delay + rate_2030s", data = df_train)

# 적합
reg_result = reg_model.fit()
print(reg_result.summary())

                            OLS Regression Results                            
Dep. Variable:                 target   R-squared:                       0.007
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     1.788
Date:                Thu, 21 Mar 2024   Prob (F-statistic):              0.168
Time:                        00:14:59   Log-Likelihood:                -8800.2
No. Observations:                 511   AIC:                         1.761e+04
Df Residuals:                     508   BIC:                         1.762e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   1.077e+07   3.31e+06      3.255      0.0

In [28]:
# rate_delay 제거
reg_model = smf.ols(formula = "target ~ rate_2030s", data = df_train)

# 적합
reg_result = reg_model.fit()
print(reg_result.summary())

                            OLS Regression Results                            
Dep. Variable:                 target   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     1.617
Date:                Thu, 21 Mar 2024   Prob (F-statistic):              0.204
Time:                        00:15:29   Log-Likelihood:                -8801.2
No. Observations:                 511   AIC:                         1.761e+04
Df Residuals:                     509   BIC:                         1.761e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   1.202e+07   3.19e+06      3.771      0.0

- 유의한 변수가 없으므로 회귀모델이 의미가 없다.